In [1]:
import sys
import os
from importlib import reload

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import re

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [9]:
from collections import Counter

In [11]:
# Проверяем, выполняется ли код на Kaggle
is_kaggle = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

if is_kaggle:
    # Проверка, существует ли уже клонированный репозиторий
    if not os.path.exists('/kaggle/working/smf'):
        !git clone https://github.com/kogriv/smf.git
    else:
        print("Репозиторий уже клонирован.")
else:
    print("Запуск на локальной машине, клонирование не требуется.")

Запуск на локальной машине, клонирование не требуется.


In [13]:
try:
    # Путь для локальной машины (определяем корень проекта)
    project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    src_path = os.path.join(project_root, "src")

    # Проверяем, существует ли путь (т.е. файл "src" должен быть в корне проекта)
    if os.path.exists(src_path):
        sys.path.append(src_path)
        print("Local machine: Путь к 'src' добавлен.")
    else:
        raise FileNotFoundError(f"Path {src_path} not found on local machine.")

except:
    try:
        # Путь для Kaggle
        kaggle_src_path = '/kaggle/working/smf/src'
        if os.path.exists(kaggle_src_path):
            sys.path.append(kaggle_src_path)
            print("Kaggle: Путь к 'src' добавлен.")
        else:
            raise FileNotFoundError(f"Path {kaggle_src_path} not found on Kaggle.")

    except Exception as e:
        print(f"Не удалось добавить путь к 'src'. Ошибка: {e}")

Local machine: Путь к 'src' добавлен.


In [15]:
import promiss
import langs_clean

In [42]:
try:
    # Путь для локальной машины
    file_path = r'C:\0\smf\train.csv' 
    dtr = pd.read_csv(file_path, lineterminator='\n')
    print('Train data loaded from local machine')

except:
    try:
        # Путь для Kaggle
        file_path = '/kaggle/input/animation-reels-classification/train.csv'
        dtr = pd.read_csv(file_path, lineterminator='\n')
        print('Train data loaded from Kaggle')

    except Exception as e:
        print(f'Train data was not loaded. Error: {e}')

Train data loaded from local machine


In [44]:
try:
    # Путь для локальной машины
    file_path = r'C:\0\smf\test.csv' 
    dts = pd.read_csv(file_path, lineterminator='\n')
    print('Test data loaded from local machine')

except:
    try:
        # Путь для Kaggle
        file_path = '/kaggle/input/animation-reels-classification/test.csv'
        dts = pd.read_csv(file_path, lineterminator='\n')
        print('Test data loaded from Kaggle')

    except Exception as e:
        print(f'Test data was not loaded. Error: {e}')

Test data loaded from local machine


In [46]:
pd.set_option('display.max_colwidth', None)

In [48]:
pd.set_option('display.max_rows', None)

In [50]:
dtr[:1]

,date,reel_name,yt_reel_id,cartoon,url,text,seconds,is_shorts,broadcast,yt_channel_id,yt_channel_name,yt_ch_url,yt_channel_type,flag_closed,international,language
0,2023-12-30,100 подарков и только один можно оставить себе,PyGZL_fCbSc,none,https://www.youtube.com/watch?v=PyGZL_fCbSc,[100 подарков и только один можно оставить себе],1854.0,0.0,none,UCcartHVtvAUzfajflyeT_Gg,Miss Katy,https://www.youtube.com/channel/UCcartHVtvAUzfajflyeT_Gg,Блогеры,0.0,0.0,NaN


In [52]:
pd.reset_option('display.max_rows')

In [54]:
print('broadcast:       ',dtr.broadcast.unique())
print('yt_channel_type: ',dtr.yt_channel_type.unique())
print('flag_closed:     ',dtr.flag_closed.unique())
print('international:   ',dtr.international.unique())
print('is_shorts:       ',dtr.is_shorts.unique())

broadcast:        ['none' nan 'live' 'upcoming']
yt_channel_type:  ['Блогеры' 'Мультфильмы' 'Детские' 'Shorts' nan]
flag_closed:      [ 0.  1. nan]
international:    [ 0.  1. nan]
is_shorts:        [ 0. nan  1.]


In [56]:
pivot_shorts_seconds = dtr.pivot_table(
    values='seconds',
    index='is_shorts',
    aggfunc=['min', 'mean', 'max']
)
pivot_shorts_seconds.columns = ['min_seconds', 'mean_seconds', 'max_seconds']
pivot_shorts_seconds = pivot_shorts_seconds.reset_index()
pivot_shorts_seconds

,is_shorts,min_seconds,mean_seconds,max_seconds
0,0.0,0.0,2023.129919,504055.0
1,1.0,1.0,27.140457,60.0


In [58]:
dtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83411 entries, 0 to 83410
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             83411 non-null  object 
 1   reel_name        83408 non-null  object 
 2   yt_reel_id       83411 non-null  object 
 3   cartoon          83411 non-null  object 
 4   url              83411 non-null  object 
 5   text             83411 non-null  object 
 6   seconds          74653 non-null  float64
 7   is_shorts        74653 non-null  float64
 8   broadcast        74653 non-null  object 
 9   yt_channel_id    83366 non-null  object 
 10  yt_channel_name  83363 non-null  object 
 11  yt_ch_url        83363 non-null  object 
 12  yt_channel_type  83363 non-null  object 
 13  flag_closed      83363 non-null  float64
 14  international    83363 non-null  float64
 15  language         497 non-null    object 
dtypes: float64(4), object(12)
memory usage: 10.2+ MB


In [60]:
dts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55608 entries, 0 to 55607
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             55608 non-null  object 
 1   reel_name        55605 non-null  object 
 2   yt_reel_id       55608 non-null  object 
 3   url              55608 non-null  object 
 4   text             55608 non-null  object 
 5   seconds          49807 non-null  float64
 6   is_shorts        49807 non-null  float64
 7   broadcast        49807 non-null  object 
 8   yt_channel_id    55584 non-null  object 
 9   yt_channel_name  55580 non-null  object 
 10  yt_ch_url        55580 non-null  object 
 11  yt_channel_type  55580 non-null  object 
 12  flag_closed      55580 non-null  float64
 13  international    55580 non-null  float64
 14  language         337 non-null    object 
dtypes: float64(4), object(11)
memory usage: 6.4+ MB


In [62]:
dtr.yt_reel_id.nunique()

83411

In [64]:
dtr.cartoon.nunique()

45

In [66]:
dtr.cartoon.value_counts(normalize=True)[:5]

none               0.937898
Маша и медведь     0.010226
Говорящий Том      0.007061
Буба               0.006342
Кошечки собачки    0.005239
Name: cartoon, dtype: float64

In [68]:
dtr.duplicated().sum()

0

Дубликатов нет

## Пропуски

In [70]:
dtr[dtr.reel_name.isnull()]

,date,reel_name,yt_reel_id,cartoon,url,text,seconds,is_shorts,broadcast,yt_channel_id,yt_channel_name,yt_ch_url,yt_channel_type,flag_closed,international,language
15331,2024-09-07,NaN,_S1ro6h6Ils,none,https://www.youtube.com/watch?v=_S1ro6h6Ils,[],19.0,0.0,none,UCfBtfDbuAH6tZnb-Q9EzauA,LEV family SHOW RUS,https://youtube.com/channel/UCfBtfDbuAH6tZnb-Q9EzauA,Блогеры,0.0,0.0,NaN
17756,2023-01-09,NaN,daJ0drzcYz4,none,https://www.youtube.com/watch?v=daJ0drzcYz4,[] shorts shortsviral babyalive deixaolike],NaN,NaN,NaN,UCUT1XWGIx6MxWmtPsGYcBEg,Bush Baby World Пушастики,https://www.youtube.com/channel/UCUT1XWGIx6MxWmtPsGYcBEg,Мультфильмы,0.0,0.0,NaN
66766,2022-11-08,NaN,Zu_vwE48tqI,none,https://www.youtube.com/watch?v=Zu_vwE48tqI,[] shortsviral babyalive babyalive brincadeira],NaN,NaN,NaN,UCUT1XWGIx6MxWmtPsGYcBEg,Bush Baby World Пушастики,https://www.youtube.com/channel/UCUT1XWGIx6MxWmtPsGYcBEg,Мультфильмы,0.0,0.0,NaN


In [72]:
dts[dts.reel_name.isnull()]

,date,reel_name,yt_reel_id,url,text,seconds,is_shorts,broadcast,yt_channel_id,yt_channel_name,yt_ch_url,yt_channel_type,flag_closed,international,language
3945,2024-04-12,NaN,TbqhXXrqVMI,https://www.youtube.com/watch?v=TbqhXXrqVMI,[],9.0,0.0,none,UCfBtfDbuAH6tZnb-Q9EzauA,LEV family SHOW RUS,https://youtube.com/channel/UCfBtfDbuAH6tZnb-Q9EzauA,Блогеры,0.0,0.0,NaN
30847,2022-10-01,NaN,u9NcRq-Ua6Y,https://www.youtube.com/watch?v=u9NcRq-Ua6Y,[] shortsviral shorts trend],NaN,NaN,NaN,UCUT1XWGIx6MxWmtPsGYcBEg,Bush Baby World Пушастики,https://www.youtube.com/channel/UCUT1XWGIx6MxWmtPsGYcBEg,Мультфильмы,0.0,0.0,NaN
49702,2023-01-04,NaN,FUKpsV638os,https://www.youtube.com/watch?v=FUKpsV638os,[] shortsviral shorts brincadeira babyalive deixaolike],NaN,NaN,NaN,UCUT1XWGIx6MxWmtPsGYcBEg,Bush Baby World Пушастики,https://www.youtube.com/channel/UCUT1XWGIx6MxWmtPsGYcBEg,Мультфильмы,0.0,0.0,NaN


Пропуски в поле `reel_name` - в малом количестве и не принадлежат к размеченным образцам. В базовом варианте обработки - удалим их

# Определим языки

In [74]:
langs_clean.lang_distrib_train

{'ru': 0.790567,
 'en': 0.15209,
 'bg': 0.01296,
 'pl': 0.009064,
 'ja': 0.006546,
 'pt': 0.006138,
 'et': 0.005815,
 'ar': 0.003369,
 'uk': 0.003105,
 'es': 0.002805,
 'mk': 0.001247,
 'ca': 0.000935,
 'no': 0.000623,
 'hi': 0.000539,
 'id': 0.00048,
 'nl': 0.000444,
 'sv': 0.000384,
 'ko': 0.000384,
 'de': 0.000384,
 'da': 0.00036,
 'hr': 0.000288,
 'af': 0.000276,
 'fr': 0.00018,
 'ro': 0.000156,
 'tr': 0.00012,
 'it': 0.00012,
 'tl': 9.6e-05,
 'vi': 9.6e-05,
 'sl': 8.4e-05,
 'cy': 6e-05,
 'sq': 4.8e-05,
 'fi': 3.6e-05,
 'unknown': 3.6e-05,
 'so': 3.6e-05,
 'th': 3.6e-05,
 'sw': 2.4e-05,
 'lt': 2.4e-05,
 'hu': 2.4e-05,
 'el': 1.2e-05,
 'sk': 1.2e-05}

# Анализ классов (выявление дисбаланса)

In [76]:
# Преобразование категориальных данных
# label_encoder = LabelEncoder()
# dtr['cartoon_encoded'] = label_encoder.fit_transform(dtr['cartoon'])